In [1]:
import numpy as np

In [2]:
"""
ds function?

fitness array ?
"""

"""
Left:
main, 
test_functions,
test_functions_range
"""

'\nLeft:\nmain, \ntest_functions,\ntest_functions_range\n'

In [3]:
def massCalculation(fit, min_flag):
  """
  This function calculates the mass of each agent
  """

  Fmax, Fmin, Fmean = np.max(fit), np.min(fit), np.mean(fit); 
  N = fit.shape

  if Fmax == Fmin:
    M = np.ones((N,1))
  else:
    if min_flag == 1:
      best, worst = Fmin, Fmax
    else:
      best, worst =Fmax, Fmin
  
  M = (fit-worst)/(best-worst)
  M = M/sum(M)
  return M

In [4]:
def move(X, a, V):
  """This function updates the velocity and 
  position of agents.
  """
  N, dim = X.shape
  V = np.random.rand(N,dim)*V + a 
  X += V

  return X, V

In [5]:
def space_bound(X, up, low):
  """This function checks the search space boundaries for agents."""
  N,dim = X.shape
  for i in range(N): 
      Tp = X[i,:] > up
      Tm = X[i,:] < low
      X[i,:] = ((X[i,:]*(np.logical_not((Tp+Tm))))) + \
               ((np.random.rand(1,dim)*(up-low)+low)*(Tp+Tm))

  return X

In [6]:
def initialization(dim, N, up, down):
  """This function initializes the position of 
  the agents in the search space, randomly."""
  print(up)
  if up.shape[0] == 1:
    X = np.random.rand(N, dim)*(up-down)+down
  
  if up.shape[0] > 1:
    for i in range(dim):
      high = up[i]
      low = down[i]
      X[:, i] = np.random.rand(N, 1)*(high-low)+low
  
  return X

In [7]:
def evaluateF(X, F_index):
  """This function Evaluates the agents"""
  N,dim =X.shape
  fitness = np.zeros(N)
  for i in range(N):
    L=X[i] 
    fitness[i] = test_functions(L, F_index, dim)

  return fitness

In [8]:
def Gconstant(iteration , max_it):
  """This function calculates Gravitational constant"""
  alfa, G0 = 20, 100
  G = G0*np.exp(-alfa*iteration / max_it )
  return G

In [9]:
def Gfield(M,X,G,Rnorm,Rpower,ElitistCheck, iteration, max_it):
  """This function calculates the accelaration 
  of each agent in gravitational field"""
  N,dim =X.shape
  final_per=2
  if ElitistCheck == 1:
    kbest=final_per+(1-iteration/max_it)*(100-final_per)
    kbest=round(N*kbest/100)
  else:
    kbest=N

  Ms = np.sort(M)[::-1]
  ds=Ms.argsort()[::-1]

  E = np.zeros((N,dim))

  for i in range(N):
    for ii in range(kbest):
        j=ds[ii]
        if j != i:

          v=X[i]-X[j]
          R=2
          #v/np.linalg.norm(v)

        else:
          R = 2

        for k in range(dim):
            
            print(R)
            E[i,k]=E[i,k]+np.random.rand()*(M[j])*((X[j,k]-X[i,k])/(R**Rpower))
        a= E * G
  return a  

In [10]:
def GSA(F_index,N,max_it,ElitistCheck,min_flag,Rpower):
  """Gravitational Search Algorithm"""
  Rnorm =2
  low,up, dim=test_functions_range(F_index)

  X=initialization(dim,N,up,low); 
  BestChart=[]
  MeanChart=[]

  V=np.zeros((N,dim))

  for iteration in range(max_it):
      X=space_bound(X,up,low)
      fitness=evaluateF(X,F_index)
      if min_flag==1 :
        best, best_X = np.min(fitness), np.argmin(fitness)
      else:
        best, best_X = np.max(fitness), np.argmin(fitness)

      if iteration==0:
            Fbest=best
            print(fitness.shape,np.min(fitness), np.argmin(fitness))
            Lbest=X[best_X]
      if min_flag==1:
          if best<Fbest:
                Fbest=best
                Lbest=X[best_X]
      else: 
          if best>Fbest:
                Fbest=best
                Lbest=X[best_X]      
                
  BestChart.append(Fbest)
  MeanChart.append(np.mean(fitness))
  M = massCalculation(fitness, min_flag) 
  G = Gconstant(iteration, max_it) 
  a = Gfield(M,X,G,Rnorm,Rpower,ElitistCheck,iteration,max_it)
  X,V = move(X, a, V)


  return Fbest,Lbest,BestChart,MeanChart

In [11]:
def test_functions(L, F_index, dim): 

  if F_index == 1:
    fit = - 20 * np.exp(- 0.2 * np.sqrt(sum(L ** 2) / dim)) - np.exp(sum(np.cos(np.multiply(2 * np.pi,L))) / dim) + 20 + np.e

  if F_index == 2:
    fit = sum(L ** 2 - 10 * np.cos(np.multiply(2 * np.pi,L))) + 10 * dim

  if F_index == 3:
    fit=sum(100*(L[1:dim]-(L[0:dim-1]** 2))**2+(L[0:dim-1]-1)**2)

  if F_index == 4:
    fit = sum(np.multiply(- L,np.sin(np.sqrt(np.abs(L)))))

  return fit

def Ufun(x, a, k, m): 
    y=k*((x-a)** m) * (x>a)+k*((-x-a)** m)*(x<(-a))
    return y

In [12]:
def test_functions_range(F_index): 
    
    dim = 30

    if F_index == 1:
        down = np.array([-32])
        up = np.array([32])

    if F_index == 2:
        down = np.array([-5.12])
        up = np.array([5.12])

    if F_index == 3:
        down = np.array([-30])
        up = np.array([30])

    if F_index == 4:
        down = np.array([-500])
        up = np.array([500])

    
    return down,up,dim

In [13]:
N = 50
max_it=1
ElitistCheck=1
Rpower=1
min_flag=1
F_index=1

Fbest,Lbest,BestChart,MeanChart=GSA(F_index,N,max_it,ElitistCheck,min_flag,Rpower)

Streaming output truncated to the last 5000 lines.
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2

In [14]:
print(Fbest)

20.684721248038013
